# Total Variation In-Painting in CVXPY
This notebook solves the *Total Variation In-Painting* problem for images. 
<center>
<img width="360" src="data/loki512_corrupted.png"/>
</center>



In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import matplotlib.cm as cm # colormaps

## Data acquisition
As a preliminary step we need some test images. The following code downloads three images, if they are not yet present in the local subfolder `data/`. 

In [ ]:
import urllib.request
import pathlib

def get_datafiles(names, repo):
    for name in names:
        url = repo + name
        filename = pathlib.Path('data/' + name)
        if not filename.exists():
            urllib.request.urlretrieve(url, filename)
            print(f"Downloaded {filename}")
        else:
            print(f"{filename} already exists. Skipping download.")

get_datafiles(names=['loki512.png', 'loki512_corrupted.png', 'loki512color.png'], repo='https://raw.githubusercontent.com/vbonifaci/convopt/refs/heads/main/laboratorio/data/')

We also define a helper function to display pairs of images side by side. 

In [ ]:
def side_by_side_display(img1, title1, img2, title2, cmap=None):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(img1, cmap=cmap)
    ax[0].set_title(title1)
    ax[0].axis('off')
    ax[1].imshow(img2, cmap=cmap);
    ax[1].set_title(title2)
    ax[1].axis('off');

## Grayscale Images

A grayscale image is represented as an $m \times n$ matrix of intensities
$U^\mathrm{orig}$ (typically between the values $0$ and $255$).
We are given the values $U^\mathrm{orig}_{ij}$, for $(i,j) \in \mathcal K$, where
$\mathcal K \subset \{1,\ldots, m\} \times \{1, \ldots, n\}$ is the set of indices
corresponding to known pixel values.
Our job is to *in-paint* the image by guessing the missing pixel values,
*i.e.*, those with indices not in $\mathcal K$.
The reconstructed image will be represented by $U \in {\bf R}^{m \times n}$,
where $U$ matches the known pixels, *i.e.*,
$U_{ij} = U^\mathrm{orig}_{ij}$ for $(i,j) \in \mathcal K$.

The reconstruction $U$ is found by minimizing the total variation of $U$,
subject to matching the known pixel values. We will use the *$\ell_2$ total
variation*, defined as
$$\mathop{\bf tv}(U) =
\sum_{i=1}^{m-1} \sum_{j=1}^{n-1}
\left\| \left[ \begin{array}{c}
 U_{i+1,j}-U_{ij}\\ U_{i,j+1}-U_{ij} \end{array} \right] \right\|_2.$$
Note that the norm of the discretized gradient is *not* squared.

Thus the problem may be expressed as
\begin{align}
\text{minimize } & \mathop{\bf tv}(U) \\
\text{subject to } & K \odot U = K \odot U^{\textrm{corr}} \\
& U \in {\bf R}^{m \times n}
\end{align}
where $U^{\textrm{corr}}$ is the corrupted image, $K_{ij} = 1$ if $(i,j) \in \mathcal{K}$ and 0 otherwise, and $\odot$ denotes the element-wise matrix product. 
This is a problem with affine constraints and a convex objective. 

We load the original image and the corrupted image and construct the `known` matrix. Both images are displayed below. The corrupted image has the missing pixels whited out.

In [ ]:
# Load the images
u_orig = plt.imread("data/loki512.png")
u_corr = plt.imread("data/loki512_corrupted.png")
rows, cols = u_orig.shape

In [ ]:
# Construct the 'known' matrix
# known is 1 if the pixel is known,
# 0 if the pixel was corrupted.
known = np.zeros((rows, cols))
for i in range(rows):
    for j in range(cols):
         if u_orig[i, j] == u_corr[i, j]:
            known[i, j] = 1

In [ ]:
# Draw the original and corrupted images
side_by_side_display(u_orig, 'Original Image', u_corr, 'Corrupted Image', cmap='gray')

The total variation in-painting problem can be easily expressed in CVXPY. We use the solver SCS.

In [ ]:
def TV_inpainting(known):
    # Recover the original image using total variation in-painting.
    U = cp.Variable(shape=(rows, cols))
    obj = cp.Minimize(cp.tv(U))
    constraints = [cp.multiply(known, U) == cp.multiply(known, u_corr)]
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.SCS, verbose=True, max_iters=500) # Use SCS solver to solve the problem.
    print("optimal objective value: {}".format(obj.value))
    return U, prob

In [ ]:
U, prob = TV_inpainting(known)

After solving the problem, the in-painted image is stored in `U.value`. We display the in-painted image and the intensity difference between the original and in-painted images. The intensity difference is magnified by a factor of 10 so it is more visible.

In [ ]:
img_diff = 10 * np.abs(u_orig - U.value)

side_by_side_display(U.value, 'In-Painted Image', img_diff, 'Difference Image (10x intensity)', cmap='gray')

# Color Images

For color images, the in-painting problem is similar to the grayscale case. A color image is represented as an $m \times n \times 3$ matrix of RGB values
$U^\mathrm{orig}$ (typically between the values $0$ and $255$).
We are given the pixels $U^\mathrm{orig}_{ij}$, for $(i,j) \in \mathcal K$, where
$\mathcal K \subset \{1,\ldots, m\} \times \{1, \ldots, n\}$ is the set of indices
corresponding to known pixels. Each pixel $U^\mathrm{orig}_{ij}$ is a vector in ${\bf R}^3$ of RGB values.
Our job is to *in-paint* the image by guessing the missing pixels,
*i.e.*, those with indices not in $\mathcal K$.
The reconstructed image will be represented by $U \in {\bf R}^{m \times n \times 3}$,
where $U$ matches the known pixels, *i.e.*,
$U_{ij} = U^\mathrm{orig}_{ij}$ for $(i,j) \in \mathcal K$.

The reconstruction $U$ is found by minimizing the total variation of $U$,
subject to matching the known pixel values. We will use the $\ell_2$ total
variation, defined as
$$\mathop{\bf tv}(U) =
\sum_{i=1}^{m-1} \sum_{j=1}^{n-1}
\left\| \left[ \begin{array}{c}
 U_{i+1,j}-U_{ij}\\ 
 U_{i,j+1}-U_{ij} 
 \end{array} \right] \right\|_2.$$
Note that the norm of the discretized gradient is *not* squared.

We load the original image and construct the Known matrix by randomly selecting 30% of the pixels to keep and discarding the others. The original and corrupted images are displayed below. The corrupted image has the missing pixels blacked out.

In [ ]:
#np.random.seed(1)
# Load the images
u_orig = plt.imread("data/loki512color.png")
rows, cols, colors = u_orig.shape

In [ ]:
# known is 1 if the pixel is known,
# 0 if the pixel was corrupted.
# The known matrix is initialized randomly.
known = np.zeros((rows, cols, colors))
for i in range(rows):
    for j in range(cols):
        if np.random.random() > 0.7:
            for k in range(colors):
                known[i, j, k] = 1        
u_corr = known * u_orig

In [ ]:
side_by_side_display(u_orig, 'Original Image', u_corr, 'Corrupted Image')

We express the total variation color in-painting problem in CVXPY using three matrix variables (one for the red values, one for the blue values, and one for the green values). We use the solver SCS; the default solvers don't scale to this large problem.

In [ ]:
def TV_inpainting_color(known): 
    # Recover the original image using total variation in-painting.
    variables = []
    constraints = []
    for i in range(colors):
        U = cp.Variable(shape=(rows, cols))
        variables.append(U)
        constraints.append(cp.multiply(known[:, :, i], U) == cp.multiply(known[:, :, i], u_corr[:, :, i]))
    
    prob = cp.Problem(cp.Minimize(cp.tv(*variables)), constraints)
    prob.solve(verbose=True, solver=cp.SCS, max_iters=500) # limit to 500 iterations to contain the time
    print("optimal objective value: {}".format(prob.value))
    return variables, prob

In [ ]:
variables, prob = TV_inpainting_color(known)

After solving the problem, the RGB values of the in-painted image are stored in the value fields of the three variables. We display the in-painted image and the difference in RGB values at each pixel of the original and in-painted image. Though the in-painted image looks almost identical to the original image, you can see that many of the RGB values differ.

In [ ]:
rec_arr = np.zeros((rows, cols, colors))
for i in range(colors):
    rec_arr[:, :, i] = variables[i].value
rec_arr = np.clip(rec_arr, 0, 1)
img_diff = np.clip(10 * np.abs(u_orig - rec_arr), 0, 1)

side_by_side_display(rec_arr, 'In-Painted Image', img_diff, 'Difference Image (10x intensity)')

**Acknowledgment**. This notebook was taken (with minor adaptations) from CVXPY's examples. 